# Projeto 3

<b>Tema:</b> Estimar o retorno financeiro de um filme

<b>Técnica:</b> Regressão Linear

<b>Grupo:</b> Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada
___

In [96]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [97]:
movies=pd.read_csv('movies_metadata.csv', low_memory=False)
imdb=pd.read_csv('ratings_small.csv',low_memory=False)

In [98]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [99]:
df=movies.loc[:, {'id','original_title','original_language','genres','popularity','runtime','budget','revenue'}]
df.head()

,id,runtime,revenue,genres,budget,original_title,popularity,original_language
0,862,81.0,373554033.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,Toy Story,21.946943,en
1,8844,104.0,262797249.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,Jumanji,17.015539,en
2,15602,101.0,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,Grumpier Old Men,11.7129,en
3,31357,127.0,81452156.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,Waiting to Exhale,3.859495,en
4,11862,106.0,76578911.0,"[{'id': 35, 'name': 'Comedy'}]",0,Father of the Bride Part II,8.387519,en


In [100]:
#Tirando budgets diferente de 0 (budget ainda como string
df=df[df.budget != "0"]
df=df.drop([19730, 29503,35587])

In [101]:
#Transformando em float id e budget
df.id=df.id.astype('float64')
df.budget=df.budget.astype('float64')
df.dtypes

id                   float64
runtime              float64
revenue              float64
genres                object
budget               float64
original_title        object
popularity            object
original_language     object
dtype: object

In [102]:
#Ordenando em ordem crescente de acordo com o id, e filtrando valores 0 das variáveis float
df=df[(df.revenue >0) & (df.budget>0)  & (df.runtime>0)]
df=df.sort_values(['id'], ascending=[True])
df

,id,runtime,revenue,genres,budget,original_title,popularity,original_language
17,5.0,98.0,4.300000e+06,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",4000000.0,Four Rooms,9.026586,en
256,11.0,121.0,7.753980e+08,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11000000.0,Star Wars,42.149697,en
6232,12.0,100.0,9.403355e+08,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",94000000.0,Finding Nemo,25.497794,en
351,13.0,142.0,6.779454e+08,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",55000000.0,Forrest Gump,48.307194,en
2742,14.0,122.0,3.562966e+08,"[{'id': 18, 'name': 'Drama'}]",15000000.0,American Beauty,20.726578,en
896,15.0,119.0,2.321767e+07,"[{'id': 9648, 'name': 'Mystery'}, {'id': 18, '...",839727.0,Citizen Kane,15.811921,en
3786,16.0,140.0,4.003188e+07,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",12800000.0,Dancer in the Dark,10.684806,en
1460,18.0,126.0,2.639202e+08,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",90000000.0,The Fifth Element,24.30526,en
1901,19.0,153.0,6.504220e+05,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",92620000.0,Metropolis,14.487867,de
6390,22.0,143.0,6.550112e+08,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",140000000.0,Pirates of the Caribbean: The Curse of the Bla...,47.326665,en
